In [12]:
#coding=utf-8
import sys
import math


class ProgressBar(object):

    FG = '\033[38;5;{}m'
    BG = '\033[48;5;{}m'
    RESETFG = '\033[39m'
    RESETBG = '\033[49m'

    COLORS = {
        'fill': [33],
        'border': [None],
        'text': [235]
    }

    def __init__(self, width=80, **kwargs):
        self.width = width
        self.progress = 0.0

        self.colors = dict(self.COLORS)
        for k, v in kwargs.items():
            if k in self.COLORS:
                if type(v) in (int, str, type(None)):
                    v = [v]
                self.colors[k] = v

    def _progressive_color(self, n):
        colorset = self.colors[n]
        n = len(colorset)
        return colorset[int(math.ceil(self.progress * n) - 1)]

    def setfg(self, colortype):
        c = self._progressive_color(colortype)
        if c is not None:
            sys.stdout.write(self.FG.format(c))
        else:
            self.resetfg()

    def resetfg(self):
        sys.stdout.write(self.RESETFG)

    def setbg(self, colortype):
        c = self._progressive_color(colortype)
        if c is not None:
            sys.stdout.write(self.BG.format(c))
        else:
            self.resetbg()

    def resetbg(self):
        sys.stdout.write(self.RESETBG)

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, exc_type, exc_val, traceback):
        pass

    def render(self):
        # Topbar
        self.setfg('border')
        sys.stdout.write(u'▁' * self.width)
        sys.stdout.write('\n')

        # Left edge
        sys.stdout.write(u'█')

        # Contents
        self.setfg('fill')
        full_width = (self.width - 2) * self.progress
        blocks = int(full_width)
        remainder = full_width - blocks

        sys.stdout.write(u'█' * blocks)

        if remainder:
            sys.stdout.write(u' ▏▎▍▌▋▊▉█'[int(remainder * 8)])

        sys.stdout.write(u' ' * (self.width - 3 - blocks))

        # Right edge
        self.setfg('border')
        sys.stdout.write(u'█')
        sys.stdout.write('\n')

        # Bottom bar
        sys.stdout.write(u'▔' * self.width)
        sys.stdout.write('\n')

        # Interior text indicator
        percent = ' ' + str(int(self.progress * 100)) + '%'
        if blocks >= len(percent):
            sys.stdout.write('\x1b7\x1b[2G\x1b[2A')
            self.setbg('fill')
            self.setfg('text')
            sys.stdout.write(percent)
            sys.stdout.write('\x1b8')

        self.resetfg()
        self.resetbg()
        sys.stdout.flush()

    def start(self):
        self.render()

    def set(self, progress):
        """
        Set progress to a specific value and redraw.
        :param progress: Fraction of the bar to fill. Clamped to [0, 1].
        """
        self.progress = min(max(progress, 0.0), 1.0)

        # Restore cursor position and redraw
        sys.stdout.write('\x1b[?25l')
        sys.stdout.write('\x1b[3F')
        self.render()
        sys.stdout.write('\x1b[?25h')

    def tick(self, amount):
        """
        Increment progress by the specified fraction and redraw.
        """
        self.set(self.progress + amount)

In [13]:
p = ProgressBar(width=80)
p.start()
for _ in range(500):
    p.tick(1.0 / 500)
    time.sleep(0.001)

▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
█                                                                             █
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
█▏                                                                             █
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
█▎                                                                             █
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
█▍                                                                             █
▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔
▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁

In [71]:
import time
for x in range (0,5):  
    b = "Loading" + "." * x
    print (b, end="\r")
    time.sleep(1)

In [1]:
import os
import sys
import time
base = '/scratch/daemon/'
dictionary = {}
while True:
    for data_type in os.listdir(base):
        In = len(os.listdir(os.path.join(base,data_type,'input')))
        Out = len(os.listdir(os.path.join(base,data_type,'output')))
        dictionary[data_type] = {'In':In,'Out':Out}
    p = ''.join(i+'\n' for i in [str(data_type+' '+'In:'+str(val['In'])+' Out:'+str(val['Out'])) for data_type,val in dictionary.items()])
    sys.stdout.write('\r'+p)
#     sys.stdout.flush()
    time.sleep(1)

stack In:250 Out:47
image In:703 Out:2086
deconvolution In:22 Out:63
dataset In:1 Out:0
registration In:2146 Out:264
hybe In:2375 Out:28
position In:266 Out:1
stack In:251 Out:47
image In:702 Out:2086
deconvolution In:22 Out:63
dataset In:1 Out:0
registration In:2146 Out:264
hybe In:2375 Out:28
position In:266 Out:1
stack In:251 Out:47
image In:701 Out:2086
deconvolution In:22 Out:63
dataset In:1 Out:0
registration In:2146 Out:264
hybe In:2375 Out:28
position In:266 Out:1
stack In:251 Out:47
image In:701 Out:2086
deconvolution In:22 Out:63
dataset In:1 Out:0
registration In:2146 Out:264
hybe In:2375 Out:28
position In:266 Out:1
stack In:251 Out:47
image In:700 Out:2086
deconvolution In:22 Out:63
dataset In:1 Out:0
registration In:2146 Out:264
hybe In:2375 Out:28
position In:266 Out:1
stack In:251 Out:47
image In:699 Out:2086
deconvolution In:22 Out:63
dataset In:1 Out:0
registration In:2146 Out:264
hybe In:2375 Out:28
position In:266 Out:1


KeyboardInterrupt: 

In [22]:
from datetime import datetime

# dd/mm/YY H:M:S
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print(dt_string)

27/07/2020 12:59:10


In [32]:
import os
import sys
import time
from datetime import datetime
base = '/scratch/daemon/'
dictionary = {}
dtypes = ['dataset','position','hybe','registration','stack','image','deconvolution']
while True:
    for data_type in dtypes:
        In = len(os.listdir(os.path.join(base,data_type,'input')))
        Out = len(os.listdir(os.path.join(base,data_type,'output')))
        dictionary[data_type] = {'In':In,'Out':Out}
    p = ''.join(i+' ' for i in [str(data_type)+'('+str(val['Out'])+':'+str(val['In'])+')' for data_type,val in dictionary.items()])
    sys.stdout.write('\r'+str(datetime.now().strftime("%m/%d/%Y %H:%M:%S"))+' '+p)
    sys.stdout.flush()
    time.sleep(5)

08/01/2020 13:07:33 dataset(0:0) position(1:0) hybe(9:0) registration(9:0) stack(18:0) image(288:0) deconvolution(18:0) 

KeyboardInterrupt: 

In [5]:
def find_dtype(t):
    t = t.split('_')
    if t[-2]!='X':
        if t[-2]=='all':
            dtype = 'deconvolution'
        else:
            dtype = 'image'
    elif t[-3]!='X':
        if t[-3]=='DeepBlue':
            dtype = 'registration'
        else:
            dtype = 'stack'
    elif t[-4]!='X':
        dtype = 'hybe'
    elif t[-5]!='X':
        dtype = 'position'
    else:
        dtype = 'dataset'
    return dtype

In [6]:
out = {}
dtypes = ['dataset','position','hybe','registration','stack','image','deconvolution']
outcomes = ['passed','failed','started']
for dtype in dtypes:
    out[dtype] = {}
    for outcome in outcomes:
        out[dtype][outcome] = []
import os
import sys
import time
from datetime import datetime
import os
import pandas as pd
path = '/hybedata/Images/Zach/ZebraFinch/A4_2020Jun28/fishdata/'
while True:
    for fname in os.listdir(path):
        if 'flag' in fname:
            t = pd.read_csv(os.path.join(path,fname)).columns[0]
            dtype = find_dtype(fname)
            if 'Started' in t:
                out[dtype]['started'].append(fname)
            elif 'Passed' in t:
                out[dtype]['passed'].append(fname)
            elif 'Failed' in t:
                out[dtype]['failed'].append(fname)
    master_string = ['dtype'+' (Passed:Failed:Started:Not Started) ']
    master_string = []
    for dtype in dtypes:
        string = dtype+'('
        for outcome in outcomes:
            string = string+str(len(out[dtype][outcome]))+':'
        string = string+') '
        master_string.append(string)
    p = ''.join(i for i in master_string)
    sys.stdout.write('\r'+str(datetime.now().strftime("%m/%d/%Y %H:%M:%S"))+' '+p)
    sys.stdout.flush()
    time.sleep(5)

08/02/2020 11:07:59 dataset(0:0:4:) position(46:0:1022:) hybe(434:0:9178:) registration(2297:0:7315:) stack(876:0:3692:) image(56895:0:2497:) deconvolution(875:0:2557:) 

KeyboardInterrupt: 